# How to store revisions and reports

In this notebook we render to HTML notebooks on the harddrive of our instance and store them to dated subdirectories in our workspace bucket so that we can have revisions.

See also [Notebooks 101 - How not to lose data output files or collaborator edits](https://broadinstitute.zendesk.com/hc/en-us/articles/360027300571-Notebooks-101-How-not-to-lose-data-output-files-or-collaborator-edits).

# Setup

In [ ]:
library(lubridate)
library(tidyverse)

Get the Cloud Storage bucket associated with this workspace.

In [ ]:
(WORKSPACE_BUCKET <- Sys.getenv('WORKSPACE_BUCKET'))

Since we cannot programmatically get a username that would look decent in a URL instead use a stamp with resolution in seconds to hopefully avoid collisions between multiple users creating revisions at the same time.

In [ ]:
# Create a timestamp for folder of results generated today.
(TIMESTAMP <- strftime(now(), '%Y%m%d/%H%M%S'))

In [ ]:
(OWNER_EMAIL <- Sys.getenv('OWNER_EMAIL'))

In [ ]:
(REPORT_DESTINATION <- str_glue('{WORKSPACE_BUCKET}/reports/{OWNER_EMAIL}/{TIMESTAMP}/'))

# Permissions check

Check that you can write to the workspace bucket.

This also serves as a way to figure out who wrote the files to the subdirectories.

**Optional** update the message below to include some comments about this revision.

In [ ]:
system(str_glue(
    'echo {OWNER_EMAIL} about to save copies of work from {TIMESTAMP} | gsutil cp - {REPORT_DESTINATION}comment.txt'),
       intern = TRUE)

In [ ]:
system(str_glue('gsutil cat {REPORT_DESTINATION}comment.txt'), intern = TRUE)

# Save copies

This notebook is currently written to only create revisions for notebooks that have been delocalized to this particular Leonardo VM. (e.g., only the notebooks I worked on recently).

In [ ]:
list.files()

## Render notebooks to HTML

**Note**: this does not re-run them, it merely takes the currently autosaved version and converts it to HTML.

In [ ]:
map(list.files(pattern = '*.ipynb'),
   function(nb) {
       print(system(str_glue('jupyter nbconvert --to html \'{nb}\''), intern = TRUE))
   })

In [ ]:
list.files(pattern = '*.html')

In [ ]:
system(str_glue('gsutil -m cp *.html {REPORT_DESTINATION}'), intern = TRUE)

# Provenance

In [ ]:
devtools::session_info()

Copyright 2019 The Broad Institute, Inc., Verily Life Sciences, LLC
All rights reserved.

This software may be modified and distributed under the terms of the BSD license.  See the LICENSE file for details.